In [1]:
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------------------------------------
# Network parameters
# ------------------------------------------------------------------
N_ctx = 50          # cortical (CS) neurons
N_la  = 25          # lateral amygdala neurons
p     = 0.3         # connection probability
dt    = 1.0         # ms simulation time step
T     = 1000        # ms total simulation time

# STDP parameters (asymmetric exponential)
tau_plus  = 20.0    # ms
tau_minus = 20.0    # ms
A_plus    = 0.01    # LTP scale
A_minus   = 0.012   # LTD scale
w_max     = 5.0     # hard bound
w0        = 0.5     # initial weight

# ------------------------------------------------------------------
# Build connectivity mask and initial weights
# ------------------------------------------------------------------
rng = np.random.default_rng(42)
C = rng.random((N_ctx, N_la)) < p          # boolean adjacency matrix
W = w0 * C.astype(float)                   # weights (synapse = C_ij * W_ij)

# ------------------------------------------------------------------
# Neuron model: perfect integrate-and-fire with fixed threshold
# ------------------------------------------------------------------
v_la  = np.zeros(N_la)       # membrane potential (LA neurons)
v_ctx = np.zeros(N_ctx)      # membrane potential (CTX neurons)

v_reset  = 0.0
v_thresh = 1.0
tau_mem  = 20.0              # membrane time constant

# Background firing rates (Poisson)
rate_ctx = 0.005             # per ms
rate_la  = 0.003             # per ms

# ------------------------------------------------------------------
# Stimulation protocol: CS-US pairing
# ------------------------------------------------------------------
CS_onset  = [200, 500, 800]  # ms
CS_dur    = 100              # ms
CS_rate   = 0.05             # per ms (strong drive)
US_rate   = 0.08             # per ms (US input to LA)

# ------------------------------------------------------------------
# STDP helpers: eligibility traces
# ------------------------------------------------------------------
pre_trace  = np.zeros(N_ctx)
post_trace = np.zeros(N_la)

# ------------------------------------------------------------------
# Logging
# ------------------------------------------------------------------
time_axis  = np.arange(0, T, dt)
w_rec      = []              # record average weight per trial

# ------------------------------------------------------------------
# Simulation loop
# ------------------------------------------------------------------
for t in time_axis:

    # ----------------------------------------------------------------
    # 1. Poisson background + CS/US
    # ----------------------------------------------------------------
    # CS spikes
    if any(on <= t < on + CS_dur for on in CS_onset):
        cs_spk = rng.random(N_ctx) < CS_rate * dt
    else:
        cs_spk = rng.random(N_ctx) < rate_ctx * dt

    # US spikes (drive to LA)
    us_spk = rng.random(N_la) < US_rate * dt if any(on <= t < on + CS_dur for on in CS_onset) else False

    # Background LA spikes
    bg_la_spk = rng.random(N_la) < rate_la * dt

    # ----------------------------------------------------------------
    # 2. Update membrane potentials
    # ----------------------------------------------------------------
    # CTX neurons spike if CS or background
    ctx_spk = cs_spk
    v_ctx *= np.exp(-dt / tau_mem)
    v_ctx[ctx_spk] = v_reset

    # LA neurons integrate CTX input + US + background
    I_syn = (W.T @ ctx_spk.astype(float))      # sum over CTX→LA synapses
    dv = (-v_la + I_syn + us_spk.astype(float) + bg_la_spk.astype(float)) * (dt / tau_mem)
    v_la += dv
    la_spk = v_la >= v_thresh
    v_la[la_spk] = v_reset

    # ----------------------------------------------------------------
    # 3. STDP update
    # ----------------------------------------------------------------
    # Update eligibility traces
    pre_trace  *= np.exp(-dt / tau_plus)
    post_trace *= np.exp(-dt / tau_minus)

    # Pre-before-post LTP
    dW_ltp = A_plus * np.outer(ctx_spk, post_trace)
    # Post-before-pre LTD
    dW_ltd = A_minus * np.outer(pre_trace, la_spk)

    W += (dW_ltp - dW_ltd) * C              # only where synapse exists
    W = np.clip(W, 0, w_max)

    # Update traces with current spikes
    pre_trace[ctx_spk]  += 1.0
    post_trace[la_spk]  += 1.0

    # ----------------------------------------------------------------
    # 4. Recording
    # ----------------------------------------------------------------
    if int(t) % 50 == 0:
        w_rec.append(W[C].mean())   # average weight over existing synapses

# ------------------------------------------------------------------
# Plot
# ------------------------------------------------------------------
plt.figure(figsize=(4,3))
plt.plot(np.arange(0, T, 50)[:len(w_rec)], w_rec, marker='o')
plt.ylabel('Mean CTX→LA weight')
plt.xlabel('Time (ms)')
plt.title('STDP during CS-US pairing')
plt.tight_layout()
plt.show()

AttributeError: 'bool' object has no attribute 'astype'